---
## Step 0. Before starting
### 0-1. Update R
1. Download binaries (Run R-3.X.X-win.exe) from https://www.r-project.org/.
2. Run R-3.X.X-win.exe
3. Set system path.
    - Type "env" in the Start button and select "Edit the system environment variables".
    - Push "Environment Variables..." button.
    - Change "Path" variable.
4. Install IRkernel for Jupyter
    - in Anaconda prompt (Admin)
    ```
    > R
    > install.packages('IRkernel')
    > IRkernel::installspec()
    ```
    
### 0-2. How to nstall packages
**Only for the first installation**, open command window.<BR>
**\<on Win10\ as regular user>**
```
C:\Users\User>R
> install.packages("ggpubr")
```
It will ask if you want to create personal for installation. Answer with **"yes"**. Once you do so, you can install additional packages from Jupyter.

**\<on Ubuntu\>**
```
$ sudo R
> install.packages("ggpubr")
```

**\<using Jupyter\>**

In [ ]:
install.packages("magrittr")

### 0-3. Useful packages
- **ggpubr**: 'ggplot2' Based Publication Ready Plots<BR>
https://cran.r-project.org/web/packages/ggpubr/index.html<BR>
- **magrittr**: It provides a new **“pipe”-like operator**, %>%<BR>
https://cran.r-project.org/web/packages/magrittr/vignettes/magrittr.html    
- **stringr**: Simple, Consistent Wrappers for **Common String Operations**<BR>
https://www.rdocumentation.org/packages/stringr/versions/1.4.0<BR>
    used as `for (colname in str_subset(names(df), rex)){` in the function conv_str2list().
- **hablar**: Simple tools for **converting columns to new data types**. Intuitive functions for columns with missing values.<BR>
https://cran.r-project.org/web/packages/hablar/<BR>
    used as `convert(lgl(single_animal))`.
- **dplyre**: dplyr is a grammar of data manipulation, providing a consistent set of verbs that help you solve the most common data manipulation challenges.<BR>
https://dplyr.tidyverse.org/<BR>
    used as `not yet`.

### 0-4. Check R version

In [ ]:
version

---
## Step 1. Package preparation
### 1-1. Install packages

In [ ]:
# Install packages

install.packages("magrittr")
install.packages("hablar")
install.packages("ggbeeswarm")
install.packages("tidyverse")
# install.packages("dplyr")
# install.packages("stringr")
install.packages("Rmisc")
install.packages("e1071")

### 1-2. Load libraries

In [ ]:
# Load required library
library(Rmisc)
library(hablar)
library(tidyverse)
# library(dplyr)

path = "C:/Users/User/Dropbox/Jupyter/wataru/synchro_freeze"
base = "synchro_freeze.R"
filename = file.path(path, base)
source(filename)

library(magrittr)
library(ggplot2)
library(ggpubr) # ggplot2 based publication ready plots

#library(Rmisc)

---
## Step 2. Import CSV file and save as R data frame

In [ ]:
#######################################
# Load the big table into t1
# R accepts both ways to describe path.
# filename <- "C:\\Users\\User\\Desktop\\project\\summary3.csv"

path = "Z:/videos_synchrony"
base = "summary3.csv"
filename = file.path(path, base)
t1 = read.table(file=filename,header=TRUE, sep=",")
t1 = t1 %>% convert(lgl(single_animal))

# Load groups table into t2
base = "IVs.csv"
filename = file.path(path, base)
t2 = read.table(file=filename,header=TRUE, sep=",")
t2 = t2 %>% convert(lgl(single_animal))

#######################################
# merge two data frames by IDs
df <- merge(t2, t1, by=c("folder_videoname","single_animal"))
# rename of column names may be necessary
#names(df)[names(df) == "single_animal.x"] <- "single_animal"

#######################################
# Post process
# Convert strings to integer list
rex = "fz_start*|fz_end*|lagt_*"
df = conv_str2list(df, rex)

# Adjust dtype
df = df %>% convert(chr(folder_videoname,sex,familiarity,lighting,stress,comment,infusion_hpc,infusion_pfc))

# Set NA for empty cell
df[df==""]<-NA
# df[is.nan(df)] <- NA

#######################################
# Exclude records
cat("Exclude", nrow(df[df$exclude==TRUE,]), "records from", nrow(df), "records\n")
df = df[df$exclude!=TRUE,]
cat("Remaining records are", nrow(df), "\n")

#######################################
# convert tibble to data.frame
df = as.data.frame(df)

#######################################
# Display summary
dis_summary(df)

# if(which(df$single_animal.x != df$single_animal.y) == FALSE){
#     print("folder_videoname and single_animal are consistent.")
# } else {
#     print("something is wrong")
# }

#######################################
# Save DF
# You can open from RStudio
base = "data.Rda"
filename = file.path(path, base)
save(df, file=filename)

path = "C:/Users/User/Dropbox/Jupyter/wataru/synchro_freeze"
base = "data.Rda"
filename = file.path(path, base)
save(df, file=filename)


---
## Step 3. Sex and stress effects on synchronization
**Load R dataframe**

In [ ]:
# Load DF
path = "C:/Users/User/Dropbox/Jupyter/wataru/synchro_freeze"
base = "data.Rda"
filename = file.path(path, base)
load(file=filename)

#######################################
# Display summary
dis_summary(df)

**Exclude single animal, infusion, lighting and partition experiments**<BR>
It's good to check using the data frame viewer in RStudio.

In [ ]:
df.set1 <- subset(df, 
                  single_animal==FALSE & 
                  is.na(infusion_hpc) &
                  is.na(infusion_pfc) & 
                  familiarity=='familiar' &
                  lighting =='visible' & 
                  partition==FALSE &
                  exclude==FALSE)

#######################################
# Display summary
dis_summary(df.set1)

**Change order of individual variables**<BR>
    Explicitly set the order in individual valiables

In [ ]:
df.set1 <- within(df.set1, sex <- factor(sex, levels = c("male","female")))
df.set1 <- within(df.set1, stress <- factor(stress, levels = c("no_stress","stress")))

**Plot Cohen D with SEM errorbar from familiar male and female pairs**<BR>
Remaining independent variables are sex, familiarity and stress.

- **ggplot2 dot plot : Quick start guide - R software and data visualization**<BR>
http://www.sthda.com/english/wiki/ggplot2-dot-plot-quick-start-guide-r-software-and-data-visualization

- **Multiple graphs on one page (ggplot2)**<BR>
http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
- **ggplot2 まとめ: 初歩から程よいレベルまで (in Japanese)**<BR>
https://mrunadon.github.io/images/geom_kazutanR.html 
- **Beautiful plotting in R: A ggplot2 cheatsheet**<BR>
http://zevross.com/blog/2014/08/04/beautiful-plotting-in-r-a-ggplot2-cheatsheet-3/
- **Complete themes**<BR>
    https://ggplot2.tidyverse.org/reference/ggtheme.html
- **Resizing plots in the R kernel for Jupyter notebooks**<BR>
https://blog.revolutionanalytics.com/2015/09/resizing-plots-in-the-r-kernel-for-jupyter-notebooks.html        
- **GGPLOT LEGEND TITLE, POSITION AND LABELS**<BR>
 https://www.datanovia.com/en/blog/ggplot-legend-title-position-and-labels/   
- **Arrange Multiple ggplots**<BR>
    https://rpkgs.datanovia.com/ggpubr/reference/ggarrange.html
    
Other decoration for plots not used below

    # Gray plot
    # scale_fill_grey() + theme_classic()+
    # add error bar in SD
    # stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="errorbar", color="red", width=0.2)+
    # add mean red point
    # stat_summary(fun=mean, geom="point", color="red")

In [ ]:
box_plot <- function(df.input, x1_col_label, y_col_label, x2_col_label, x2_col_value, yaxis_label){
    #######################################################
    # parameters to adjust graph
    # graph width x hight
#     plot_width = 8
#     plot_hight = 6

    # dotplot
    dp_binwidth=0.2
    dp_dotsize=0.8
    
    # y limits
    ylim1 = -2
    ylim2 = 4.5
    #######################################################
    
    # get column name
    x1 <- names(df.input)[which (colnames(df.input)==x1_col_label)]
    x2 <- names(df.input)[which (colnames(df.input)==x2_col_label)]
    y <- names(df.input)[which (colnames(df.input)==y_col_label)]
    
    # set the size of final plot
    # options(repr.plot.width=plot_width, repr.plot.height=plot_hight)

    # set data frame and axes
    ggp.output <- ggplot(df.input[df.input[,x2]==x2_col_value,], aes_string(x=x1, y=y, fill=x1))+
        # add boxplot with mean and SEM (MeanSEM) or SD (MeanSD).
        # stat_summary(fun.data=MeanSEM, geom="boxplot", colour="black", width=0.6)+
        stat_summary(fun.data=MeanSD, geom="boxplot", colour="black", width=0.5)+
        # Dotplot
        geom_dotplot(binaxis='y', stackdir='center', binwidth=dp_binwidth, dotsize=dp_dotsize, fill='black')+
        # y axis label
        labs(y=yaxis_label)+
        # Limit y axis range
        ylim(c(ylim1, ylim2))+

    # Remainings are all the same. No need to modify.
        # Change filling color for box
        scale_fill_manual(values=c("#FF3333", "#3333FF"))+
        # Change theme. Look at "Complete themes" above.
        theme_bw()+
        # Control of axes
        theme(
            axis.title.x = element_blank(),
            axis.title.y = element_text(size=25,vjust=2,face='bold'),
            axis.text=element_text(size=20))+    
        # Control of legend
        theme(
            #legend.position = "none",
            legend.position = 'bottom',
            legend.direction = "horizontal",
            #legend.title = element_text(size=20,face='bold'),
            legend.title = element_blank(),
            legend.text=element_text(size=20))
    
    return(ggp.output)
}

out1_1 = box_plot(df.set1, "sex","cohen_d", "stress", "no_stress","Cohen D")
out1_2 = box_plot(df.set1, "sex","cohen_d", "stress", "stress","Cohen D")

# plot the two plots
# set the size of final plot, width x hight
plot_width = 8
plot_hight = 6
options(repr.plot.width=plot_width, repr.plot.height=plot_hight)

ggarrange(out1_1, out1_2, ncol=2)

**Export the plot as an eps file**

In [ ]:
# Output eps file on desktop
path = "C:/Users/User/Desktop"
base = "plot.eps"
filename = file.path(path, base)

ggsave(filename, device=cairo_ps,width=plot_width, height=plot_height)

**1-sample t-test**
- R - Mean, Median and Mode
https://www.tutorialspoint.com/r/r_mean_median_mode.htm

- 13.3 T-test: t.test()
https://bookdown.org/ndphillips/YaRrr/t-test-t-test.html
    


In [ ]:
one_sample_t_test <- function(df.input, y_col, x1_col, x1_value, x2_col, x2_value){
#############################################################
# 1 sample t-test for (y_col), where
#     (x1_col) = x1_value
#     (x2_col) = x2_value
# hypothetical value (mu = 0)
#############################################################
    # get index for each column name
    #    you can use either forms work
#     x1 <- which (colnames(df.input)==x1_col)
#     x2 <- which (colnames(df.input)==x2_col)
#     y <- which (colnames(df.input)==y_col)
    x1 <- names(df.input)[which (colnames(df.input)==x1_col)]
    x2 <- names(df.input)[which (colnames(df.input)==x2_col)]
    y <- names(df.input)[which (colnames(df.input)==y_col)]
    
    # Extract data frame to vector
    temp = df.input[,y][df.input[,x1]==x1_value & df.input[,x2]==x2_value]
    
    cat(paste0('### (',x1_value,", \t",x2_value, ") \t###\t"))
    temp.test <- t.test(temp, mu=0)
    cat("p-value: ",temp.test[["p.value"]],"\n")
    # detailed information
    # print(temp.test)

    return(temp.test)
}
for (sex in c("male","female")){
    for (stress in c("stress", "no_stress")){
        temp = one_sample_t_test(df.set1,"cohen_d", "sex",sex,"stress",stress)
    }
}


**2-sample t-test**

In [ ]:
two_sample_t_test <- function(df.input, y_col, xa1_col, xa1_value, xa2_col, xa2_value,xb1_col, xb1_value, xb2_col, xb2_value){
#############################################################
# 2 sample t-test for (y_col), between
#     (xa1_col) = xa1_value and (xa2_col) = xa2_value
#     (xb1_col) = xb1_value and (xb2_col) = xb2_value
#############################################################
    # get index for each column name
    xa1 <- names(df.input)[which (colnames(df.input)==xa1_col)]
    xa2 <- names(df.input)[which (colnames(df.input)==xa2_col)]
    xb1 <- names(df.input)[which (colnames(df.input)==xb1_col)]
    xb2 <- names(df.input)[which (colnames(df.input)==xb2_col)]
    y <- names(df.input)[which (colnames(df.input)==y_col)]
    
    # Extract data frame
    temp.a = df.input[,y][df.input[,xa1]==xa1_value & df.input[,xa2]==xa2_value]
    temp.b = df.input[,y][df.input[,xb1]==xb1_value & df.input[,xb2]==xb2_value]
    
    cat(paste0('### (',xa1_value,", \t",xa2_value, ") \tvs. (", xb1_value, ", \t", xb2_value, ") \t###\t"))
    temp.test <- t.test(temp.a, temp.b)
    cat("p-value: ",temp.test[["p.value"]],"\n")
    # detailed information
    #print(temp.test)

    return(temp.test)
}

for (sex1 in c("male", "female")){
    for (sex2 in c("male", "female")){
        for (stress1 in c("stress","no_stress")){
            for (stress2 in c("stress","no_stress")){
                temp = two_sample_t_test(df.set1, "cohen_d","sex",sex1,"stress",stress1,"sex",sex2,"stress",stress2)
}}}}




---
## Step 4. Sex and stress effects on freezing
**collapse sub1 and sub2 freezing into the same column with cohen_d**

In [ ]:
# Create data frame
df.freeze = data.frame(sex=character(),stress=character(),freeze=double(),cohen_d=double())

for (sex in c("male", "female")){
    for (stress in c("stress","no_stress")){
        # Append rows from onset
        df.freeze = df.freeze %>% add_row(
            "freeze" = unlist(df.set1['fz_sub1'][df.set1$sex==sex & df.set1$stress==stress,]),
            "cohen_d" = unlist(df.set1['cohen_d'][df.set1$sex==sex & df.set1$stress==stress,]),
            "sex" = sex, "stress" = stress)
    }
}

for (sex in c("male", "female")){
    for (stress in c("stress","no_stress")){
        # Append rows from onset
        df.freeze = df.freeze %>% add_row(
            "freeze" = unlist(df.set1['fz_sub2'][df.set1$sex==sex & df.set1$stress==stress,]),
            "cohen_d" = unlist(df.set1['cohen_d'][df.set1$sex==sex & df.set1$stress==stress,]),
            "sex" = sex, "stress" = stress)
    }
}


# Explicitly set the order in individual valiables
df.freeze <- within(df.freeze, sex <- factor(sex, levels = c("male","female")))
df.freeze <- within(df.freeze, stress <- factor(stress, levels = c("no_stress","stress")))

#######################################
# Display summary
dis_summary(df.freeze)

**plot the graphs**

In [ ]:
box_plot <- function(df.input, x1_col_label, y_col_label, x2_col_label, x2_col_value, yaxis_label){
    #######################################################
    # parameters to adjust graph
    # graph width x hight
#     plot_width = 8
#     plot_hight = 6

    # dotplot
    dp_binwidth=0.7
    dp_dotsize=3
    
    # y limits
    ylim1 = -2
    ylim2 = 90
    #######################################################
    
    # get column name
    x1 <- names(df.input)[which (colnames(df.input)==x1_col_label)]
    x2 <- names(df.input)[which (colnames(df.input)==x2_col_label)]
    y <- names(df.input)[which (colnames(df.input)==y_col_label)]
    
    # set the size of final plot
    # options(repr.plot.width=plot_width, repr.plot.height=plot_hight)

    # set data frame and axes
    ggp.output <- ggplot(df.input[df.input[,x2]==x2_col_value,], aes_string(x=x1, y=y, fill=x1))+
        # add boxplot with mean and SEM (MeanSEM) or SD (MeanSD).
        # stat_summary(fun.data=MeanSEM, geom="boxplot", colour="black", width=0.6)+
        stat_summary(fun.data=MeanSD, geom="boxplot", colour="black", width=0.5)+
        # Dotplot
        geom_dotplot(binaxis='y', stackdir='center', binwidth=dp_binwidth, dotsize=dp_dotsize, fill='black')+
        # y axis label
        labs(y=yaxis_label)+
        # Limit y axis range
        ylim(c(ylim1, ylim2))+

    # Remainings are all the same. No need to modify.
        # Change filling color for box
        scale_fill_manual(values=c("#FF3333", "#3333FF"))+
        # Change theme. Look at "Complete themes" above.
        theme_bw()+
        # Control of axes
        theme(
            axis.title.x = element_blank(),
            axis.title.y = element_text(size=25,vjust=2,face='bold'),
            axis.text=element_text(size=20))+    
        # Control of legend
        theme(
            #legend.position = "none",
            legend.position = 'bottom',
            legend.direction = "horizontal",
            #legend.title = element_text(size=20,face='bold'),
            legend.title = element_blank(),
            legend.text=element_text(size=20))
    
    return(ggp.output)
}

out2_1 = box_plot(df.freeze, "sex","freeze", "stress", "no_stress","Freezing (%)")
out2_2 = box_plot(df.freeze, "sex","freeze", "stress", "stress","Freezing (%)")

# plot the two plots
# set the size of final plot, width x hight
plot_width = 8
plot_hight = 6
options(repr.plot.width=plot_width, repr.plot.height=plot_hight)

ggarrange(out2_1, out2_2, ncol=2)

**Export the plot as an eps file**

In [ ]:
# Output eps file on desktop
path = "C:/Users/User/Desktop"
base = "plot.eps"
filename = file.path(path, base)

ggsave(filename, device=cairo_ps,width=plot_width, height=plot_height)

**2-sample t-test**

In [ ]:
two_sample_t_test <- function(df.input, y_col, xa1_col, xa1_value, xa2_col, xa2_value,xb1_col, xb1_value, xb2_col, xb2_value){
#############################################################
# 2 sample t-test for (y_col), between
#     (xa1_col) = xa1_value and (xa2_col) = xa2_value
#     (xb1_col) = xb1_value and (xb2_col) = xb2_value
#############################################################
    # get index for each column name
    xa1 <- names(df.input)[which (colnames(df.input)==xa1_col)]
    xa2 <- names(df.input)[which (colnames(df.input)==xa2_col)]
    xb1 <- names(df.input)[which (colnames(df.input)==xb1_col)]
    xb2 <- names(df.input)[which (colnames(df.input)==xb2_col)]
    y <- names(df.input)[which (colnames(df.input)==y_col)]
    
    # Extract data frame
    temp.a = df.input[,y][df.input[,xa1]==xa1_value & df.input[,xa2]==xa2_value]
    temp.b = df.input[,y][df.input[,xb1]==xb1_value & df.input[,xb2]==xb2_value]
    
    cat(paste0('### (',xa1_value,", \t",xa2_value, ") \tvs. (", xb1_value, ", \t", xb2_value, ") \t###\t"))
    temp.test <- t.test(temp.a, temp.b)
    cat("p-value: ",temp.test[["p.value"]],"\n")
    # detailed information
    #print(temp.test)
    
    return(temp.test)
}

for (sex1 in c("male", "female")){
    for (sex2 in c("male", "female")){
        for (stress1 in c("stress","no_stress")){
            for (stress2 in c("stress","no_stress")){
                temp = two_sample_t_test(df.freeze, "freeze","sex",sex1,"stress",stress1,"sex",sex2,"stress",stress2)
}}}}


---
**Examine the output from 2-sample t-test**<BR>
The class of output is 'htest', and complecated!

In [ ]:
# temp is the output from the previous cell
# print out the test
temp
# list variable names in the temp
names(temp)
# print lass of the temp output
class(temp)
# stracture of the temp output
str(temp)
# summary of variables of the temp output
summary(temp)

**Naking the output**<BR>
    Output is a list of multiple lists

In [ ]:
# Show the content of parameter
temp['parameter']
temp[2]

# Extract the inside list
temp[['parameter']]
temp[[2]]
temp$parameter

# Show the content of "df" value in the inside list
temp[['parameter']]['df']
temp[[2]][1]

# Extract value from the 
temp[['parameter']][['df']]
temp[[2]][[1]]

**You can even add new value**

In [ ]:
temp[['parameter']]['df1'] = 1

---
## Summary graphs

In [ ]:
# set the size of final plot
plot_height = 6
plot_width = 12
options(repr.plot.width=plot_width, repr.plot.height=plot_height)

out1_1a = annotate_figure(out1_1,top = text_grob("no stress", face = "bold", size = 20))
out1_2a = annotate_figure(out1_2,top = text_grob("stress", face = "bold", size = 20))
out2_1a = annotate_figure(out2_1,top = text_grob("no stress", face = "bold", size = 20))
out2_2a = annotate_figure(out2_2,top = text_grob("stress", face = "bold", size = 20))

b1 = ggarrange(out1_1a, out1_2a, ncol=2)
b1 = annotate_figure(b1,top = text_grob("Synchronization", face = "bold", size = 25))
b2 = ggarrange(out2_1a, out2_2a, ncol=2)
b2 = annotate_figure(b2,top = text_grob("Freezing", face = "bold", size = 25))

ggarrange(b1, b2, ncol=2)

**Export the plot as an eps file**

In [ ]:
# Output eps file on desktop
path = "C:/Users/User/Desktop"
base = "plot.eps"
filename = file.path(path, base)

ggsave(filename, device=cairo_ps,width=plot_width, height=plot_height)

**Two-way ANOVA test**

In [ ]:
temp.aov = aov(cohen_d ~ freeze + stress, data = df.freeze)
summary(temp.aov)

temp.aov = aov(cohen_d ~ freeze + stress, data = df.freeze[df.freeze$sex == "male",])
summary(temp.aov)

temp.aov = aov(cohen_d ~ freeze + stress, data = df.freeze[df.freeze$sex == "female",])
summary(temp.aov)

---
## Step 5. Correlation between freezing levels and synchronization
**plot the scatter plot**

In [ ]:
scatter_plot <- function(df.input, x1_col, y_col, x2_col, x3_col, x3_value, xaxis_label, yaxis_label){
    #######################################################
    # parameters to adjust graph
    # graph width x hight
    plot_width = 8
    plot_hight = 6

    # dotplot
#     dp_binwidth=0.7
#     dp_dotsize=3
   
    point_size = 3
    
    
    # y limits
    ylim1 = -3
    ylim2 = 5
    #######################################################
    
    # get column name
    x1 <- names(df.input)[which (colnames(df.input)==x1_col)]
    x2 <- names(df.input)[which (colnames(df.input)==x2_col)]
    x3 <- names(df.input)[which (colnames(df.input)==x3_col)]
    y <- names(df.input)[which (colnames(df.input)==y_col)]

    # set the size of final plot
    options(repr.plot.width=plot_width, repr.plot.height=plot_hight)
        
    ggplot(df.input[df.input[,x3]==x3_value,], aes_string(x=x1,y=y,fill=x2,color=x2))+
        geom_point(size=point_size)+
        geom_point(size=point_size,shape=1,colour="black")+
        geom_smooth(method=lm)+

        # x axis label
        labs(x=xaxis_label)+
        # y axis label
        labs(y=yaxis_label)+
        # Limit y axis range
        ylim(c(ylim1, ylim2))+

    # Remainings are all the same. No need to modify.
        # Change filling color for box
        scale_fill_manual(values=c("#FF3333", "#3333FF"))+
        scale_colour_manual(values=c("#FF3333", "#3333FF"))+
        # Change theme. Look at "Complete themes" above.
        theme_bw()+
        # Control of axes
        theme(
            # axis.title.x = element_blank(),
            axis.title = element_text(size=25,vjust=2,face='bold'),
            axis.text=element_text(size=20))+    
        # Control of legend
        theme(
            #legend.position = "none",
            legend.position = 'bottom',
            legend.direction = "horizontal",
            #legend.title = element_text(size=20,face='bold'),
            legend.title = element_blank(),
            legend.text=element_text(size=20))

}


out3_1 = scatter_plot(df.freeze, "freeze", "cohen_d", "stress", "sex","male", "Freezeing (%)", "Cohen D")
out3_2 = scatter_plot(df.freeze, "freeze", "cohen_d", "stress", "sex","female", "Freezeing (%)", "Cohen D")

In [ ]:
plot_height = 6
plot_width = 12
# set the size of final plot
options(repr.plot.width=plot_width, repr.plot.height=plot_height)
ggarrange(out3_1, out3_2, ncol=2)

**Export the plot as an eps file**

In [ ]:
# Output eps file on desktop
path = "C:/Users/User/Desktop"
base = "plot.eps"
filename = file.path(path, base)

ggsave(filename, device=cairo_ps,width=plot_width, height=plot_height)

In [ ]:
my_data <- read.csv(file.choose())

**Linear Regression**<BR>
http://r-statistics.co/Linear-Regression.html

- **Correlation:**
Correlation is a statistical measure that suggests the level of linear dependence between two variables, that occur in pair – just like what we have here in speed and dist. Correlation can take values between -1 to +1. If we observe for every instance where speed increases, the distance also increases along with it, then there is a high positive correlation between them and therefore the correlation between them will be closer to 1. The opposite is true for an inverse relationship, in which case, the correlation between the variables will be close to -1.

A value closer to 0 suggests a weak relationship between the variables. **A low correlation (-0.2 < x < 0.2)** probably suggests that much of variation of the response variable (Y) is unexplained by the predictor (X), in which case, we should probably look for better explanatory variables.

- **The linear correlation coefficient (Pearson’s r)** is just the standardized slope of a simple linear regression line.<BR>
    https://sebastianraschka.com/faq/docs/pearson-r-vs-linear-regr.html
    

**Test the entire data**

In [ ]:
# Less correlation in the entirly collapsed data set

# calculate correlation between freeze and cohen_d
cat("Correlation: ",cor(df.freeze$freeze, df.freeze$cohen_d),"\n")

# Modeling the linear model
temp.linearmod = lm(cohen_d ~ freeze, data=df.freeze)

# Details
temp.summary = summary(temp.linearmod)
print(temp.summary)

**It looks like the following perason command is the digest version**

In [ ]:
cor.test(df.freeze$freeze, df.freeze$cohen_d, method = 'pearson')

**Test each group**
- R Tip: How to Pass a formula to lm**<BR>
http://www.win-vector.com/blog/2018/09/r-tip-how-to-pass-a-formula-to-lm/

In [ ]:
# Effects of sex and stress on the correlation

comp_cor <- function(df.input, pred_col, resp_col, lim1_col, lim1_value, lim2_col, lim2_value){
    
    # get column name
    pred_col <- names(df.input)[which (colnames(df.input)==pred_col)]
    resp_col <- names(df.input)[which (colnames(df.input)==resp_col)]
    lim1_col <- names(df.input)[which (colnames(df.input)==lim1_col)]
    lim2_col <- names(df.input)[which (colnames(df.input)==lim2_col)]    

    # Extract data frame
    temp = df.input[df.input[,lim1_col]==lim1_value & df.input[,lim2_col]==lim2_value,]
    
    # compute correlations
    temp.cor = cor(temp[,pred_col], temp[,resp_col])
    
    # estimate the linear model
    # create formula to fit
    f = paste(resp_col, pred_col, sep = " ~ ")
    #cat("formula: ", f, ", ")
    
    # create model based on the created formula
    temp.linearmod = lm(f, data=temp)
    temp.summary = summary(temp.linearmod)
    
    # output in brief
    cat(paste0('### (',lim1_value,", ",lim2_value, ") ###, "))
    cat("correlation: ", temp.cor, ", ")
    cat("intercept_p-value: ", temp.summary[['coefficients']]['(Intercept)','Pr(>|t|)'], ", ")
    cat("freeze_p-value: ", temp.summary[['coefficients']]['freeze','Pr(>|t|)'], "\n")

    # output full
#     cat("correlation: ", temp.cor, "\n")
#     cat("formula: ", f, "\n")
#     print(temp.summary)
    
    temp.list = list("cor"=temp.cor, "f"=f, "linearmod"=temp.linearmod, "summary"=temp.summary)
    return(temp.list)
}
    
for (sex1 in c("male", "female")){
        for (stress1 in c("stress","no_stress")){
                temp = comp_cor(df.freeze, "freeze","cohen_d","sex",sex1,"stress",stress1)
}}

---
## 2. Visualize the distribution of lagtime for onset and offset of freezing
### 2-1. Collapse the all cells in lagt_* columns and create a new DF of two columns (lagtime, type)

In [ ]:
# Create working df and collapse cells in columns of "lagt_*"
rex = "lagt_*"

# Initialize dataframe
df.work = data.frame(matrix(ncol = 4, nrow = 1))
colnames(df.work) = str_subset(names(df), rex)

# Collapse each column
for (colname in str_subset(names(df), rex)){
    w = c() # working vector
    for (i in c(1:nrow(df))){
        # w = c(w, as.integer(unlist(df[i,colname])))
        w = c(w, unlist(df[i,colname]))
    }
    df.work[[1,colname]] = list(w)
}

# Mearge both directions, s1_s2 and s2_s1
for (colname in c("lagt_start", "lagt_end")){
    df.work[,colname] = NA # Append empty column
    # w = c(as.integer(unlist(df.work[1, paste0(colname, "_s1_s2")])),as.integer(unlist(df.work[1,paste0(colname, "_s2_s1")])))
    w = c(unlist(df.work[1, paste0(colname, "_s1_s2")]),unlist(df.work[1,paste0(colname, "_s2_s1")]))
    df.work[[1,colname]]=list(w)
}

# Create New DF, df.test of lagtime and type (onset or offset)
df.lagtime = data.frame("lagtime" = unlist(df.work[1,"lagt_start"]), "type" = "onset")
# Append rows from onset
df.lagtime = df.lagtime %>% add_row("lagtime" = unlist(df.work[1,"lagt_end"]), "type" = "offset")

# convert frame number to sec
df.lagtime[,"lagtime"] = df.lagtime[,"lagtime"] / 4

### 2-2. Visualize

In [ ]:
# Plot the distribution
library(ggplot2)
library(ggbeeswarm)

ggplot(df.lagtime,aes(type,lagtime)) +
 geom_boxplot() +
 geom_quasirandom(alpha = 0.2) +
 theme_bw()

# # Colored Histogram with Different Number of Bins
sub.lagtime = df.lagtime[df.lagtime$type=='onset',]
ggplot(sub.lagtime,aes(x=lagtime)) +
    geom_histogram(binwidth=0.2)

sub.lagtime = df.lagtime[df.lagtime$type=='offset',]
ggplot(sub.lagtime,aes(x=lagtime)) +
    geom_histogram(binwidth=0.2)

# hist(subTest$lagtime, breaks=seq(,,10),col="blue", xlim=c(-250,250), ylim=c(0,3000))
#, breaks=seq(-50,50,1), col="blue", xlim=c(-50,50), ylim=c(0,60))
# hist(subTest$lagtime, breaks=seq(-50,50,1), col="blue", xlim=c(-50,50), ylim=c(0,60))

# subTest = test[test$type=='offset',]
# hist(subTest$lagtime, breaks=seq(-50,50,1), col="blue", xlim=c(-50,50), ylim=c(0,60))


# ### (Option) Export a plot as EPS file ##############################################
# # Change the plot line
# setEPS()
# postscript("whatever.eps")
# plot(rnorm(100), main="Hey Some Data")
# dev.off()
# #####################################################################################


### 2-3. Test Coefficients of Variation from multiple samples
https://cran.r-project.org/web/packages/cvequality/vignettes/how_to_test_CVs.html

**Need to install packages**

`install.packages("ggbeeswarm")`<BR>
`install.packages("cvequality")`


In [ ]:
install.packages("cvequality")

In [ ]:
# Load required library
library(cvequality)

test1 <- with(df.lagtime,asymptotic_test(lagtime,type))
test1

In [ ]:
test2 <- with(df.lagtime,mslr_test(nr = 1e4, lagtime,type))
test2

---
# r-plotmaking
       20200501 am
#### Required package
- **Hmisc package**: Contains many functions useful for data analysis, high-level graphics, utility operations, functions for computing sample size and power, importing and annotating datasets, imputing missing values, advanced table making, variable clustering, character string manipulation, conversion of R objects to LaTeX and html code, and recoding variables.<BR> https://cran.r-project.org/web/packages/Hmisc/index.html

In [ ]:
# Install packages
install.packages("Hmisc")

#load necessary libraries and packages
library(ggplot2)
library(Hmisc)
library(readxl)

## 1. plotting LTP data using means and SEM

In [ ]:
# 1. plotting LTP data using means and SEM

# 1-1. Read csv file named LTPtest
path = "C:/Users/User/Dropbox/Jupyter/alexei/r_plotmaking_data"
base = "LTPtest.csv"
filename = file.path(path, base)
LTP <- read.csv(filename)

# 1-2. Make line plot
p<-ggplot(LTP, aes(x=time, y=means))+geom_line(stat="identity",color="red") + geom_point(color="red") + geom_errorbar(aes(ymin=means-SEM, ymax=means+SEM))
print(p)

## 2. making dot plot with error bars using table with individual data points

In [ ]:
# 2. making dot plot with error bars using table with individual data points

# 2-1. import data from excel as df1 using "Import Dataset" function in Environment tab, one column containing datapoints, other columns containing independent variables
base = "synchfreeze_aggregated.xlsx"
filename = file.path(path, base)
df1 <- read_excel("C:/Users/User/Dropbox/Jupyter/alexei/r_plotmaking_data/synchfreeze_aggregated.xlsx", 
  sheet = "for Rplot")

# 2-2. plotting cohen distance with "condition" as factor
p <- ggplot(df1, aes(x=condition, y=cohen)) + 
  geom_dotplot(binaxis='y', stackdir='center')

p + stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), 
                 geom="errorbar", color="red", width=0.2) +
  stat_summary(fun.y=mean, geom="point", color="red")

## 3. plotting paired comparisons of freezing between mice tested alone or in pairs

In [ ]:
# 3. plotting paired comparisons of freezing between mice tested alone or in pairs

# 3-1. load excel file
# library(readxl)
base = "freezing_ind_vs_pair_for_Rplot.xlsm"
filename = file.path(path, base)
df1 <- read_excel(filename, sheet = "females")

# 3-2. plot pair comparisons
p<- ggplot(df1, aes(x=condition, y=freezing, group=animal, color=animal)) + 
  geom_line() + geom_point()
print(p)

## (Option) Export a plot as EPS file

In [ ]:
### (Option) Export a plot as EPS file ##############################################
# Change the plot line
setEPS()
postscript("whatever.eps")
# plot(rnorm(100), main="Hey Some Data")
print(p)
dev.off()
#####################################################################################

---
# One-Sample Wilcoxon Signed Rank Test in R
http://www.sthda.com/english/wiki/one-sample-wilcoxon-signed-rank-test-in-r

In [ ]:
# We want to know, if the average of the data differs from mu (two-tailed test).

# One-sample wilcoxon test
res <- wilcox.test(df.lagtime$V1, mu = 44)
# Printing the results
res

---
# Boxplot for the distribution of lagtime for each animal pair.

In [ ]:
# Raw data for lag-time
# "s" stands for onset and "e" stands for offset of freezing
d1 <- c( 0,    -24,    3,    0,   16,    8,    9,   -3,    5,    4,   -3,   -1,  -2 )
e1 <- c("f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s", "f1s") 

d2 <- c(   0,   -9,    0,    0,   13,    0,    0,    4,    2,   -1,   -8,   3)
e2 <- c("f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e", "f1e") 

d3 <- c(14,-1,9,8,-7,0,-8,0,0,-18,0)
e3 <- c("f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s","f2_1s")

d4 <- c(     13,      10,      -3,     -11,      -9,       1,      -3,       3,     -18,      -6,       4,     0)
e4 <- c("f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e", "f2_1e") 

d5 <- c(3,4,0,4,3,1,0,-8,-4,5,11)
e5 <- c("f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s","f2_2s")

d6 <- c(3,1,0,1,1,0,0,-9,-12,3,-2)
e6 <- c("f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e","f2_2e")

d7 <- c(-3,-1,-7,11,-11,3,-13,-2)
e7 <- c("f3_1s","f3_1s","f3_1s","f3_1s","f3_1s","f3_1s","f3_1s","f3_1s")

d8 <- c(-7,0,-2,2,-7,2,-6,0)
e8 <- c("f3_1e","f3_1e","f3_1e","f3_1e","f3_1e","f3_1e","f3_1e","f3_1e")

d9 <- c(3,-12,4,-2,5)
e9 <- c("f3_2s","f3_2s","f3_2s","f3_2s","f3_2s")

d10 <- c(-1,-3,0,-12)
e10 <- c("f3_2e","f3_2e","f3_2e","f3_2e")

d11 <- c(-11,-14,11,16,-57,19)
e11 <- c("f4_1s","f4_1s","f4_1s","f4_1s","f4_1s","f4_1s")

d12 <- c(-16,-10,-25,-1,0)
e12 <- c("f4_1e","f4_1e","f4_1e","f4_1e","f4_1e")

d13 <- c(3,-4,-18,18)
e13 <- c("f4_2s","f4_2s","f4_2s","f4_2s")

d14 <- c(8,3,-13,16,0)
e14 <- c("f4_2e","f4_2e","f4_2e","f4_2e","f4_2e")

# Concatenate the data
d <- c(d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14)
e <- c(e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14)

# Create data frame
mydata <- data.frame(d,e)
# Add column names
names(mydata) <- c("s1_s2","pair")

mydata

In [ ]:
# Boxplot for the distribution of lag-time
library(ggplot2)
library(ggbeeswarm)

ggplot(mydata,aes(pair,s1_s2)) + geom_boxplot() + geom_quasirandom(alpha = 0.9) + theme_bw()

### (Option) Export a plot as EPS file ##############################################
# Change the plot line
setEPS()
postscript("whatever.eps")
plot(rnorm(100), main="Hey Some Data")
dev.off()
#####################################################################################

---

# R version

In [ ]:
version

---
# Read csv file and test correlation
The csv file is generated by MATLAB code

[READING IN DATA FROM AN EXTERNAL FILE | R LEARNING MODULES](https://stats.idre.ucla.edu/r/modules/reading-in-data-from-an-external-file/)

In [ ]:
test <- read.table('D:\\wataru\\Recording_Analysis\\Bases_dmPFC-BLA\\2017-12-19_vm81a_base\\myFile.txt', sep = ",")

In [ ]:
ccf(test[,1], test[,3], lag = 200000, ylim = range(-1,1), type="correlation")

In [ ]:
testTS <- ts(test)

In [ ]:
length(testTS)
str(testTS)
class(testTS)
names(testTS)
testTS

In [ ]:
test <- read.table('D:\\wataru\\Recording_Analysis\\Bases_dmPFC-BLA\\2017-12-19_vm81a_base\\myFile.txt', sep = ",")

In [ ]:
data (sales)  # parts of Example 11.2.2 from Brockwell and Davies (1991).
sal <- diff (sales)
led <- diff(lead)
ccf (led, sal, lag = 20, ylim = range(-1,1), type="o")

In [ ]:
set.seed(123)
x = arima.sim(model=list(0.2, 0, 0.5), n = 100)
y = arima.sim(model=list(0.4, 0, 0.4), n = 100)
ccf(x, y, type="correlation")

In [ ]:
readClipboard()

In [ ]:
# setwd("D:/wataru/Recording_Analysis/Bases_dmPFC-BLA")
# theta <- scan('test.txt')
# plot(theta)

theta <- scan('D:\\wataru\\Recording_Analysis\\Bases_dmPFC-BLA\\2017-12-19_vm81a_base\\test.txt')
plot(theta)

---
# Data Types
https://www.statmethods.net/input/datatypes.html

In [ ]:
######################################################
# vectors
a <- c(1,2,5.3,6,-2,4) # numeric vector
b <- c("one","two","three") # character vector
c <- c(TRUE,TRUE,TRUE,FALSE,TRUE,FALSE) #logical vector

# Identify rows, columns or elements using subscripts.
a[4]
a[c(2,4)]

######################################################
# matrix
# generates 5 x 4 numeric matrix 
y<-matrix(1:20, nrow=5,ncol=4)
# another example
cells <- c(1,26,24,68)
rnames <- c("R1", "R2")
cnames <- c("C1", "C2") 
mymatrix <- matrix(cells, nrow=2, ncol=2, byrow=TRUE,
  dimnames=list(rnames, cnames))

# Identify rows, columns or elements using subscripts.
x[,4] # 4th column of matrix
x[3,] # 3rd row of matrix 
x[2:4,1:3] # rows 2,3,4 of columns 1,2,3

######################################################
# Data Frames
# A data frame is more general than a matrix, in that different columns can have different
# modes (numeric, character, factor, etc.). This is similar to SAS and SPSS datasets.

d <- c(1,2,3,4)
e <- c("red", "white", "red", NA)
f <- c(TRUE,TRUE,TRUE,FALSE)
mydata <- data.frame(d,e,f)
names(mydata) <- c("ID","Color","Passed") # variable names

# Identify rows, columns or elements using subscripts.
mydata[2:3] # columns 3,4,5 of data frame
mydata[c("ID","Passed")] # columns ID and Age from data frame
mydata$Color # variable x1 in the data frame
mydata[1,3]

######################################################
# The ls() function returns a vector listing lists all the objects (vectors, data frames, etc) in your current workspace.
ls()

# Remove these three objects
rm("first_name", "last_name", "new_df")
 
# Or remove objects listed in a vector
rm(list = c("first_name", "last_name", "new_df"))
 
# Or remove all files from your workspace
rm(list = ls())
 
# Or remove vectors programmatically.  Delete objects with underscore in name
rm(list = ls()[grepl("_", ls())])

######################################################
# Lists
# An ordered collection of objects (components). A list allows you to gather a variety of 
# (possibly unrelated) objects under one name.
# example of a list with 4 components - 
# a string, a numeric vector, a matrix, and a scaler 

w <- list(name="Fred", mynumbers=a, mymatrix=y, age=5.3)

# example of a list containing two lists
# It looks concatenate the two lists
v <- c(w,w)

# Identify elements of a list using the [[]] convention.
mylist[[2]] # 2nd component of the list
mylist[["mynumbers"]] # component named mynumbers in list



######################################################
# Factors
# Tell R that a variable is nominal by making it a factor. The factor stores the nominal
# values as a vector of integers in the range [ 1... k ] (where k is the number of unique 
# values in the nominal variable), and an internal vector of character strings (the original 
# values) mapped to these integers.

# variable gender with 20 "male" entries and 
# 30 "female" entries 
gender <- c(rep("male",20), rep("female", 30)) 
gender <- factor(gender) 
# stores gender as 20 1s and 30 2s and associates
# 1=female, 2=male internally (alphabetically)
# R now treats gender as a nominal variable 
summary(gender)
